In [46]:
# each instance of this class stores a single fittness rule
class Fitness_Rule :  
    # takes a string which containes the rule and builds the rule
    def __init__(self, line) :
        self.option_list = []
        line_list = line.split('#')
        for element in line_list :
            if element.count(':') == 0 :
                self.option_list.append(element)
            else :
                self.option_list.append(element[:element.find(':')])
                self.fitnes_value = float(element[element.find(':')+2:])    
                
    # print overload
    def __str__(self) :
        out = ""
        for element in self.option_list :
            out += element
            out += " ** "
        out = out[:-3]
        out += " : " + str(self.fitnes_value)
        return out
        
    # takes an option dictionariy and returns the cost of this rule
    def get_partial_fitness(self, option_activation) :
        applicable = 1
        for x in self.option_list :
            applicable = applicable and option_activation[x]
        return applicable * self.fitnes_value        

    
# contains all fittness rules
class Fitness_Modell :
    # takes a string containing fittness rules and adds them to the rule list
    def add_fitness_rules(self, fittness_input) :
        for line in fittness_input :
            self.fittness_rule_list.append(Fitness_Rule(line))
            

    # takes a list containing fittness rules and builds the rule list
    def __init__(self) :
        self.fittness_rule_list = []
        
                        
    # print overload
    def __str__(self) :
        out = ""
        for element in self.fittness_rule_list :
            out += element.__str__() + "\n"
        out = out[:-2]
        return out
    
    # takes an option activation dictionary and calculates the fitness(or cost) value
    def calculate_fitness(self, option_activation) : 
        fittness = 0.0
        for element in self.fittness_rule_list :
            fittness += element.get_partial_fitness(option_activation)
        return fittness



# takes a dictionary and re
#def calculate fitness()

In [81]:
# an instance of this class stores all constrains
# TODO: optimize validity check
class Constraint_Model :
        
    def __init__(self) :
        self.constraint_list = []
        self.global_tabu_list = []
        self.mandatory_activation = []
                 
    # input line(string) list
    def build_model(self, dimacs_input) :
        self.constraint_list = []
        self.global_tabu_list = []
        self.mandatory_activation = []
        tmp_constraint = []
        
        dimacs_input = [x.strip() for x in dimacs_input]
        for line in dimacs_input :
            # fill constraint list 
            if len(line) > 0 and line[0] != 'c' and line[0] != 'p' :
                line_list = line.split(' ')
                for element in line_list :
                    if element == '0' and len(element) > 0 :
                        self.constraint_list.append(tmp_constraint)
                        tmp_constraint = []
                    elif len(element) > 0 :
                        print("|"+str(element)+"|")
                        tmp_constraint.append(int(element))      
            # initialize tabu list and mandatory activation
            elif len(line) > 0 and line[0] == 'p' :
                line_list = line.split(' ')
                self.global_tabu_list = [False] * (int(line_list[2]) + 1)
                self.mandatory_activation = [None] * (int(line_list[2]) + 1)
                
                
    # print function            
    def __str__ (self) :
        out = "Constraints:\n"
        for a in self.constraint_list :
            out += str(a) + "-"+ str(len(a)) + ", "
        
        out += "\n\nGLobal Tabu List:\n"
        out += str(self.global_tabu_list)
        
        out += "\n\nMandatory Activation List:\n"
        out += str(self.mandatory_activation)
           
        return out
        
    
    # builds the global tabu list and sets the activation values 
    def build_global_tabu_list(self) :
        new_constraint_list = []
        for constraint in self.constraint_list :  
            if len(constraint) == 1 :       
                self.global_tabu_list[abs(constraint[0])] = True      
                self.mandatory_activation[abs(constraint[0])] = constraint[0] > 0    
            else :
                new_constraint_list.append(constraint)
                
        self.constraint_list = new_constraint_list
#        for constraint in self.constraint_list :
#            if len(constraint) == 2 :
    
    
    # TODO: optimize !!!!!!
    def check_validity (self, activation_arry) :
        for constraint in self.constraint_list :
            if not self.check_partial_validity(constraint, activation_arry) :
                return False
        return True
    
    # TODO: optimize
    def check_partial_validity (self, constraint, activation_arry) :
        for x in constraint :
            if x > 0 :
                if activation_arry[x] :
                    return True
            else :
                 if not activation_arry[x * (-1)] :
                    return True   
        return False    

In [82]:
def linear_regression(n) :
    import itertools
    combinations = list(map(list, itertools.product([0, 1], repeat=n)))
    return(combinations)

def parse_xml_to_dimacs(file):
    var_map = [] #

    final_string = "" #
    variables_string = "" #
    p_line = "p cnf " #
    clauses = "" #
    current_options = [] # 
    optional_false_elements = []
    clauses_2D = []

    from lxml import etree
    xml_rules = etree.parse(open(file, 'r'))
    root = xml_rules.getroot()

    # first iteration to generate the dimacs variables
    for child in root :
        for subchild in child :
            for element in subchild : 
                if element.tag == 'name':
                    var_map.append(element.text)

    # generate variables
    for el in var_map:
        variables_string += "c " + str(1+var_map.index(el)) + " " + el + "\n"

    # generate rules
    for child in root :
        for subchild in child :
            for element in subchild :

                # temporary save current element and excluded options
                if element.tag == 'name':
                    current_options.append(element.text)
                if element.tag == 'excludedOptions' :
                    for option in element :
                        current_options.append(option.text)

                # clear temp if 'otional' tag is set to true
                if element.tag == 'optional' :
                    if element.text != 'False' :
                        current_options = []

            # if there are choices to do generate all combinations at once
            if len(current_options) > 0 :
                ###########################################################################
                # adding all combinations of rules if they are not already in 
                ###########################################################################
                # current_name, current_options
                #
                # print("Inside combinations loop", str(len(current_options)))

                if len(current_options) == 1 :
                    #print("exact one element")
                    for element in current_options:
                        #print("1")
                        tmp=[]
                        tmp.append(1+var_map.index(element))
                        clauses_2D.append(tmp)

                if len(current_options) > 1 :
                    ####################################################current_options = current_options.sort()
                    combinations = linear_regression(len(current_options))
                    for lst in combinations :
                        #print(str(len(current_options)))
                        #print(len(combinations))
                        tmp_list = []
                        for el, el2 in zip(lst, current_options) :
                            if el :
                                tmp_list.append(1+var_map.index(el2))
                            else :
                                tmp_list.append((1+var_map.index(el2))*(-1))

                        # do not put duplicate elements in list
                        tmp_list.sort()
                        is_in_list = False
                        for item in clauses_2D:
                            if item == tmp_list:
                                is_in_list = True
                        if not is_in_list :
                            clauses_2D.append(tmp_list)

            current_options = []

    p_line += str(len(var_map))+" "+str(len(clauses_2D))+"\n"

    #print(clauses_2D)
    for line in clauses_2D :
        for item in line :
            clauses += str(item)+" "
        clauses += "0\n"

    # und zack fertig .... dimacs
    final_string = variables_string+p_line+clauses
    print(final_string)
    return final_string.split('\n')

In [83]:
class Wrapper :
    
    def __init__(self, path_to_folder, project_name) :
        from os import listdir
        from os.path import isfile, join
        
        
        self.fitness_model = Fitness_Modell()
        self.constraint_model = Constraint_Model()
        
        
        
       
        files = [join(path_to_folder, f) for f in listdir(path_to_folder) if isfile(join(path_to_folder, f))]
        
        for file in files:
             # -------- Fitness Model -----------
            if file.count(project_name) > 0 and (file.count('feature') > 0 or file.count('interactions') > 0) :
                with open(file, 'r') as f:
                    self.fitness_model.add_fitness_rules(f.readlines())
            
             # ------ Constraint Model ---------
            if file.count(project_name) > 0 and file.count('.xml') > 0 :
                self.constraint_model.build_model(parse_xml_to_dimacs(file))
                print(parse_xml_to_dimacs(file))
            if file.count(project_name) > 0 and file.count('.dimacs') > 0 :
                with open(file, 'r') as f:
                    self.constraint_model.build_model(f.readlines())
                    
            # analyze constraints
            self.constraint_model.build_global_tabu_list()
    

In [84]:
testinstanz = Wrapper('./project_public_2/', 'busy')
#testinstanz = Wrapper('./project_public_1/', 'h264')
print(testinstanz.fitness_model)
print(testinstanz.constraint_model)


|1691|
|-4244|
|-1691|
|611|
|-1691|
|612|
|-1692|
|-4245|
|-1692|
|-611|
|-1692|
|-612|
|-1693|
|-4246|
|-1694|
|-4247|
|-1694|
|1089|
|-1694|
|1090|
|-1695|
|-4248|
|-1695|
|-1089|
|-1695|
|-1090|
|-1696|
|-4249|
|-1697|
|-4250|
|-1697|
|-907|
|-1697|
|-908|
|-1698|
|-4251|
|-1698|
|-907|
|-1698|
|-908|
|-1699|
|-4252|
|-1700|
|-4253|
|-1700|
|129|
|-1700|
|130|
|-1701|
|-4254|
|-1701|
|-129|
|-1701|
|-130|
|-1702|
|-4255|
|-1703|
|-4256|
|-1703|
|-909|
|-1703|
|-910|
|-1704|
|-4257|
|-1704|
|-909|
|-1704|
|-910|
|-1705|
|-4258|
|-1706|
|-4259|
|-1706|
|1091|
|-1706|
|1092|
|-1707|
|-4260|
|-1707|
|-1091|
|-1707|
|-1092|
|-1708|
|-4261|
|-1709|
|-4262|
|-1709|
|-1177|
|-1709|
|-1178|
|-1710|
|-4263|
|-1711|
|-4264|
|-1711|
|-875|
|-1711|
|-876|
|-1712|
|-4265|
|-1713|
|-4266|
|-1713|
|-811|
|-1713|
|-812|
|-1714|
|-4267|
|-1715|
|-4268|
|-1715|
|189|
|-1715|
|190|
|-1716|
|-4269|
|-1716|
|-189|
|-1716|
|-190|
|-1717|
|-4270|
|-777|
|189|
|190|
|-778|
|189|
|190|
|-777|
|-778|
|-189|


|1927|
|-1928|
|-487|
|-1928|
|-488|
|-1927|
|487|
|488|
|1928|
|-4481|
|-1928|
|-365|
|-1928|
|-366|
|-1929|
|-4482|
|-293|
|487|
|488|
|-294|
|487|
|488|
|-293|
|-294|
|-487|
|488|
|-1930|
|-487|
|-1930|
|-488|
|487|
|488|
|1930|
|-4483|
|-1930|
|-487|
|-293|
|-1930|
|-487|
|-294|
|-1931|
|1930|
|-1931|
|-487|
|-1931|
|-488|
|-1930|
|487|
|488|
|1931|
|-4484|
|-1931|
|-293|
|-1931|
|-294|
|-1932|
|-4485|
|-1933|
|-4486|
|-1933|
|753|
|-1933|
|754|
|-1934|
|-4487|
|-1934|
|-753|
|-1934|
|-754|
|-1935|
|-4488|
|-1936|
|-4489|
|-1936|
|-847|
|-1936|
|-848|
|-1937|
|-4490|
|-1937|
|-847|
|-1937|
|-848|
|-1938|
|-4491|
|-1939|
|-4492|
|-1939|
|1383|
|-1939|
|1384|
|-1940|
|-4493|
|-1940|
|-1383|
|-1940|
|-1384|
|-1941|
|-4494|
|-1603|
|1091|
|1092|
|-1604|
|1091|
|1092|
|-1603|
|-1604|
|-1091|
|1092|
|-1942|
|-1091|
|-1942|
|-1092|
|1091|
|1092|
|1942|
|-4495|
|-1942|
|-1091|
|-1603|
|-1942|
|-1091|
|-1604|
|-1943|
|1942|
|-1943|
|-1091|
|-1943|
|-1092|
|-1942|
|1091|
|1092|
|1943|
|-4496

|-413|
|223|
|224|
|-414|
|491|
|492|
|-414|
|223|
|492|
|-414|
|491|
|224|
|-414|
|223|
|224|
|-413|
|-414|
|-491|
|-223|
|492|
|-413|
|-414|
|-491|
|-223|
|224|
|-2043|
|-491|
|-223|
|-2043|
|-492|
|-224|
|491|
|492|
|2043|
|491|
|224|
|2043|
|223|
|492|
|2043|
|223|
|224|
|2043|
|-4596|
|-2043|
|-491|
|-223|
|-413|
|491|
|-2043|
|-491|
|-223|
|-413|
|223|
|-2043|
|-491|
|-223|
|-491|
|-223|
|413|
|-2043|
|-491|
|-223|
|-414|
|492|
|-2043|
|-491|
|-223|
|-414|
|224|
|-2043|
|-491|
|-223|
|-492|
|-224|
|414|
|-2044|
|2043|
|-2044|
|-491|
|-223|
|-2044|
|-492|
|-224|
|-2043|
|491|
|492|
|2044|
|-2043|
|491|
|224|
|2044|
|-2043|
|223|
|492|
|2044|
|-2043|
|223|
|224|
|2044|
|-4597|
|-2044|
|-413|
|-2044|
|-414|
|-2045|
|-4598|
|-1185|
|491|
|492|
|-1185|
|413|
|492|
|-1185|
|491|
|414|
|-1185|
|413|
|414|
|-1186|
|491|
|492|
|-1186|
|413|
|492|
|-1186|
|491|
|414|
|-1186|
|413|
|414|
|-1185|
|-1186|
|-491|
|-413|
|492|
|-1185|
|-1186|
|-491|
|-413|
|414|
|-2046|
|-491|
|-413|
|-2046|
|-

|-2184|
|1396|
|-2185|
|-4738|
|-2185|
|-1395|
|-2185|
|-1396|
|-2186|
|-4739|
|-433|
|1395|
|567|
|637|
|1396|
|568|
|638|
|-434|
|1395|
|567|
|637|
|1396|
|568|
|638|
|-433|
|-434|
|-1395|
|1396|
|568|
|638|
|-433|
|-434|
|-567|
|1396|
|568|
|638|
|-433|
|-434|
|-637|
|1396|
|568|
|638|
|-2187|
|-1395|
|-2187|
|-567|
|-2187|
|-637|
|-2187|
|-1396|
|-2187|
|-568|
|-2187|
|-638|
|1395|
|567|
|637|
|1396|
|568|
|638|
|2187|
|-4740|
|-2187|
|-1395|
|-433|
|1395|
|567|
|637|
|-2187|
|-567|
|-433|
|1395|
|567|
|637|
|-2187|
|-637|
|-433|
|1395|
|567|
|637|
|-2187|
|-1395|
|-1395|
|433|
|-2187|
|-1395|
|-567|
|433|
|-2187|
|-1395|
|-637|
|433|
|-2187|
|-567|
|-1395|
|433|
|-2187|
|-567|
|-567|
|433|
|-2187|
|-567|
|-637|
|433|
|-2187|
|-637|
|-1395|
|433|
|-2187|
|-637|
|-567|
|433|
|-2187|
|-637|
|-637|
|433|
|-2187|
|-1395|
|-434|
|1396|
|568|
|638|
|-2187|
|-567|
|-434|
|1396|
|568|
|638|
|-2187|
|-637|
|-434|
|1396|
|568|
|638|
|-2187|
|-1395|
|-1396|
|434|
|-2187|
|-1395|
|-568|
|434|


|-2364|
|-1317|
|-822|
|1318|
|-2364|
|-1317|
|-1318|
|822|
|-2365|
|2364|
|-2365|
|-1317|
|-2365|
|-1318|
|-2364|
|1317|
|1318|
|2365|
|-4918|
|-2365|
|-821|
|-2365|
|-822|
|-2366|
|-4919|
|-2367|
|-4920|
|-2367|
|653|
|-2367|
|654|
|-2368|
|-4921|
|-2368|
|-653|
|-2368|
|-654|
|-2369|
|-4922|
|-2370|
|-4923|
|-2370|
|301|
|-2370|
|302|
|-2371|
|-4924|
|-2371|
|-301|
|-2371|
|-302|
|-2372|
|-4925|
|-2373|
|-4926|
|-2373|
|1415|
|-2373|
|1416|
|-2374|
|-4927|
|-2374|
|-1415|
|-2374|
|-1416|
|-2375|
|-4928|
|-2376|
|-4929|
|-2376|
|815|
|-2376|
|816|
|-2377|
|-4930|
|-2377|
|-815|
|-2377|
|-816|
|-2378|
|-4931|
|-2379|
|-4932|
|-2379|
|331|
|-2379|
|332|
|-2380|
|-4933|
|-2380|
|-331|
|-2380|
|-332|
|-2381|
|-4934|
|-609|
|331|
|332|
|-610|
|331|
|332|
|-609|
|-610|
|-331|
|332|
|-2382|
|-331|
|-2382|
|-332|
|331|
|332|
|2382|
|-4935|
|-2382|
|-331|
|-609|
|331|
|-2382|
|-331|
|-331|
|609|
|-2382|
|-331|
|-610|
|332|
|-2382|
|-331|
|-332|
|610|
|-2383|
|2382|
|-2383|
|-331|
|-2383|
|-33

...

|-3857|
|-6410|
|-3857|
|-1607|
|-3857|
|-1608|
|-3858|
|-6411|
|-63|
|1091|
|1092|
|-64|
|1091|
|1092|
|-63|
|-64|
|-1091|
|1092|
|-3859|
|-1091|
|-3859|
|-1092|
|1091|
|1092|
|3859|
|-6412|
|-3859|
|-1091|
|-63|
|1091|
|-3859|
|-1091|
|-1091|
|63|
|-3859|
|-1091|
|-64|
|1092|
|-3859|
|-1091|
|-1092|
|64|
|-3860|
|3859|
|-3860|
|-1091|
|-3860|
|-1092|
|-3859|
|1091|
|1092|
|3860|
|-6413|
|-3860|
|-63|
|-3860|
|-64|
|-3861|
|-6414|
|-3862|
|-6415|
|-3862|
|595|
|-3862|
|596|
|-3863|
|-6416|
|-3863|
|-595|
|-3863|
|-596|
|-3864|
|-6417|
|-197|
|595|
|596|
|-198|
|595|
|596|
|-197|
|-198|
|-595|
|596|
|-3865|
|-595|
|-3865|
|-596|
|595|
|596|
|3865|
|-6418|
|-3865|
|-595|
|-197|
|595|
|-3865|
|-595|
|-595|
|197|
|-3865|
|-595|
|-198|
|596|
|-3865|
|-595|
|-596|
|198|
|-3866|
|3865|
|-3866|
|-595|
|-3866|
|-596|
|-3865|
|595|
|596|
|3866|
|-6419|
|-3866|
|-197|
|-3866|
|-198|
|-3867|
|-6420|
|-277|
|595|
|596|
|-278|
|595|
|596|
|-277|
|-278|
|-595|
|596|
|-3868|
|-595|
|-3868|
|-596|
|59

|1295|
|1296|
|-558|
|1295|
|1296|
|-557|
|-558|
|-1295|
|1296|
|-3994|
|-1295|
|-3994|
|-1296|
|1295|
|1296|
|3994|
|-6547|
|-3994|
|-1295|
|-557|
|1295|
|-3994|
|-1295|
|-1295|
|557|
|-3994|
|-1295|
|-558|
|1296|
|-3994|
|-1295|
|-1296|
|558|
|-3995|
|3994|
|-3995|
|-1295|
|-3995|
|-1296|
|-3994|
|1295|
|1296|
|3995|
|-6548|
|-3995|
|-557|
|-3995|
|-558|
|-3996|
|-6549|
|-1541|
|1295|
|1296|
|-1542|
|1295|
|1296|
|-1541|
|-1542|
|-1295|
|1296|
|-3997|
|-1295|
|-3997|
|-1296|
|1295|
|1296|
|3997|
|-6550|
|-3997|
|-1295|
|-1541|
|1295|
|-3997|
|-1295|
|-1295|
|1541|
|-3997|
|-1295|
|-1542|
|1296|
|-3997|
|-1295|
|-1296|
|1542|
|-3998|
|3997|
|-3998|
|-1295|
|-3998|
|-1296|
|-3997|
|1295|
|1296|
|3998|
|-6551|
|-3998|
|-1541|
|-3998|
|-1542|
|-3999|
|-6552|
|-383|
|1645|
|1295|
|1646|
|1296|
|-384|
|1645|
|1295|
|1646|
|1296|
|-383|
|-384|
|-1645|
|1646|
|1296|
|-383|
|-384|
|-1295|
|1646|
|1296|
|-4000|
|-1645|
|-4000|
|-1295|
|-4000|
|-1646|
|-4000|
|-1296|
|1645|
|1295|
|1646|
|1296|

|1221|
|-4123|
|-1221|
|-1221|
|1193|
|-4123|
|-1221|
|-1194|
|1222|
|-4123|
|-1221|
|-1222|
|1194|
|-4124|
|4123|
|-4124|
|-1221|
|-4124|
|-1222|
|-4123|
|1221|
|1222|
|4124|
|-6677|
|-4124|
|-1193|
|-4124|
|-1194|
|-4125|
|-6678|
|-4126|
|-6679|
|-4126|
|299|
|-4126|
|300|
|-4127|
|-6680|
|-4127|
|-299|
|-4127|
|-300|
|-4128|
|-6681|
|-4129|
|227|
|-227|
|4129|
|-6682|
|228|
|-228|
|6682|
|-4129|
|637|
|638|
|-6682|
|637|
|638|
|-4129|
|-6682|
|-637|
|638|
|-4130|
|-6683|
|-4130|
|637|
|-4130|
|638|
|-4131|
|-6684|
|-4131|
|-637|
|4129|
|-4131|
|-4129|
|637|
|-4131|
|-638|
|6682|
|-4131|
|-6682|
|638|
|-4132|
|-6685|
|-677|
|637|
|638|
|-678|
|637|
|638|
|-677|
|-678|
|-637|
|638|
|-4133|
|-637|
|-4133|
|-638|
|637|
|638|
|4133|
|-6686|
|-4133|
|-637|
|-677|
|637|
|-4133|
|-637|
|-637|
|677|
|-4133|
|-637|
|-678|
|638|
|-4133|
|-637|
|-638|
|678|
|-4134|
|4133|
|-4134|
|-637|
|-4134|
|-638|
|-4133|
|637|
|638|
|4134|
|-6687|
|-4134|
|-677|
|-4134|
|-678|
|-4135|
|-6688|
|-245|
|677|


|466|
|-1565|
|1566|
|-715|
|716|
|-23|
|24|
|-1057|
|1058|
|-1527|
|1528|
|-723|
|724|
|-655|
|656|
|-1223|
|1224|
|-1291|
|1292|
|-1303|
|1304|
|-223|
|224|
|-1119|
|1120|
|-295|
|296|
|-1231|
|1232|
|-1581|
|1582|
|-1511|
|1512|
|-957|
|958|
|-551|
|552|
|-47|
|48|
|-205|
|206|
|-1015|
|1016|
|-1009|
|1010|
|-1613|
|1614|
|-1653|
|1654|
|-327|
|328|
|-349|
|350|
|-669|
|670|
|-1637|
|1638|
|-49|
|50|
|-877|
|878|
|-549|
|550|
|-1101|
|1102|
|-855|
|856|
|-1253|
|1254|
|-1001|
|1002|
|-1165|
|1166|
|-1293|
|1294|
|-1639|
|1640|
|-1217|
|1218|
|-1203|
|1204|
|-967|
|968|
|-843|
|844|
|-607|
|608|
|-169|
|170|
|-1655|
|1656|
|-1473|
|1474|
|-487|
|488|
|-1209|
|1210|
|-1455|
|1456|
|-517|
|518|
|-13|
|14|
|-365|
|366|
|-293|
|294|
|-753|
|754|
|-847|
|848|
|-1603|
|1604|
|-419|
|420|
|-1307|
|1308|
|-1385|
|1386|
|-1103|
|1104|
|-1471|
|1472|
|-1081|
|1082|
|-415|
|416|
|-973|
|974|
|-727|
|728|
|-1379|
|1380|
|-857|
|858|
|-1299|
|1300|
|-81|
|82|
|-1053|
|1054|
|-1431|
|1432|
|-1503|

|-440|
|441|
|-442|
|443|
|-444|
|445|
|-446|
|447|
|-448|
|449|
|-450|
|451|
|-452|
|453|
|-454|
|455|
|-456|
|457|
|-458|
|459|
|-460|
|461|
|-462|
|463|
|-464|
|465|
|-466|
|467|
|-468|
|469|
|-470|
|471|
|-472|
|473|
|-474|
|475|
|-476|
|477|
|-478|
|479|
|-480|
|481|
|-482|
|483|
|-484|
|485|
|-486|
|487|
|-488|
|489|
|-490|
|491|
|-492|
|493|
|-494|
|495|
|-496|
|497|
|-498|
|499|
|-500|
|501|
|-502|
|503|
|-504|
|505|
|-506|
|507|
|-508|
|509|
|-510|
|511|
|-512|
|513|
|-514|
|515|
|-516|
|517|
|-518|
|519|
|-520|
|521|
|-522|
|523|
|-524|
|525|
|-526|
|527|
|-528|
|529|
|-530|
|531|
|-532|
|533|
|-534|
|535|
|-536|
|537|
|-538|
|539|
|-540|
|541|
|-542|
|543|
|-544|
|545|
|-546|
|547|
|-548|
|549|
|-550|
|551|
|-552|
|553|
|-554|
|555|
|-556|
|557|
|-558|
|559|
|-560|
|561|
|-562|
|563|
|-564|
|565|
|-566|
|567|
|-568|
|569|
|-570|
|571|
|-572|
|573|
|-574|
|575|
|-576|
|577|
|-578|
|579|
|-580|
|581|
|-582|
|583|
|-584|
|585|
|-586|
|587|
|-588|
|589|
|-590|
|591|
|-592|
|593|